In [28]:
from matplotlib.pyplot import imread

# -*- coding: utf-8 -*-
"""
Created on Saturday Nov 15 15:34 2025

@author: 100yearsahead


Bleaching Presence Detection
Target variable: Percent_Bleaching

"""

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import os


path = "./coral-reef-global-bleaching"
filename_read = os.path.join(path, "coral_whole.csv")

df = pd.read_csv(filename_read)

# Removed only locations and labels, no real data touched
# df.drop("Ocean_Name", axis=1, inplace=True)
# df.drop("Country_Name", axis=1, inplace=True)
# df.drop("Sample_ID", axis=1, inplace=True)
# df.drop("Date_Year", axis=1, inplace=True)
# df.drop("Bleaching_Level", axis=1, inplace=True)
# df.drop("Realm_Name", axis=1, inplace=True)
# # #Percent_Cover is not a best predictor and also contain 30% of its fields as null.
# # # For the sake of bigger dataset this feature is dropped
# df.drop("Percent_Cover", axis=1, inplace=True)
# # df.drop("ClimSST", inplace=True, axis=1)
# df.drop("Exposure", inplace=True, axis=1)
# # df.drop("Temperature_Maximum", inplace=True, axis=1)

# label_encoder = LabelEncoder()
# df["Exposure"] = label_encoder.fit_transform(df["Exposure"])

# These features were taken into account that data is nonlinear
# df = df[["Distance_to_Shore", "Temperature_Mean", "Turbidity", "TSA", "Depth_m", "Percent_Bleaching"]]

# These features were taken into account that data is linear
#df = df[['Cyclone_Frequency', 'Depth_m', 'ClimSST', 'Distance_to_Shore', 'Turbidity', 'TSA', 'Temperature_Mean', 'Percent_Bleaching']]

df  = df.drop(columns=['Sample_ID', 'Percent_Cover'])
df.dropna(inplace=True)

df.info()
print(df)


<class 'pandas.core.frame.DataFrame'>
Index: 32714 entries, 0 to 35044
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Cyclone_Frequency    32714 non-null  float64
 1   Depth_m              32714 non-null  float64
 2   ClimSST              32714 non-null  float64
 3   Ocean_Name           32714 non-null  object 
 4   Country_Name         32714 non-null  object 
 5   Distance_to_Shore    32714 non-null  float64
 6   Exposure             32714 non-null  object 
 7   Turbidity            32714 non-null  float64
 8   Date_Year            32714 non-null  int64  
 9   Bleaching_Level      32714 non-null  object 
 10  Temperature_Maximum  32714 non-null  float64
 11  SSTA                 32714 non-null  float64
 12  TSA                  32714 non-null  float64
 13  Percent_Bleaching    32714 non-null  float64
 14  Temperature_Mean     32714 non-null  float64
 15  Realm_Name           32714 non-null  obje

In [29]:
import pandas as pd
import numpy as np
from scipy.stats import skew, boxcox
from sklearn.preprocessing import PowerTransformer


numeric_cols = df.select_dtypes(include=[np.number]).columns
skewness_results = {}

for col in numeric_cols:
    clean_data = df[col].dropna()
    if len(clean_data) > 0:
        skew_val = skew(clean_data)
        skewness_results[col] = {
            'skewness': skew_val,
            'skew_type': 'Right (Positive)' if skew_val > 0.5 else 'Left (Negative)' if skew_val < -0.5 else 'Approximately Symmetric',

        }

right_skewed = {col: data for col, data in skewness_results.items() if data['skewness'] > 0.5}
left_skewed = {col: data for col, data in skewness_results.items() if data['skewness'] < -0.5}

print(left_skewed.keys())
print(right_skewed.keys())
for col_right in right_skewed.keys():
    df[col_right] = df[col_right].replace(0, 1e-6)
    df[col_right], lam = boxcox(df[col_right])

for col_left in left_skewed.keys():
    pt = PowerTransformer(method='yeo-johnson')
    df[[col_left]] = pt.fit_transform(df[[col_left]])

skewness_df = pd.DataFrame(skewness_results).T
skewness_df = skewness_df.sort_values('skewness', key=abs, ascending=False)
skewness_df[:-1]

dict_keys(['ClimSST', 'TSA', 'Temperature_Mean'])
dict_keys(['Cyclone_Frequency', 'Depth_m', 'Distance_to_Shore', 'Turbidity', 'Temperature_Maximum', 'Percent_Bleaching'])


C:\Users\zemsk\anaconda3\Lib\site-packages\numpy\_core\_methods.py:197: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
C:\Users\zemsk\anaconda3\Lib\site-packages\numpy\_core\_methods.py:208: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)


,skewness,skew_type
Distance_to_Shore,9.390477,Right (Positive)
Turbidity,3.848336,Right (Positive)
Percent_Bleaching,2.969849,Right (Positive)
ClimSST,-1.640544,Left (Negative)
Temperature_Mean,-1.320474,Left (Negative)
Depth_m,1.272371,Right (Positive)
TSA,-1.013456,Left (Negative)
Temperature_Maximum,0.930379,Right (Positive)
Cyclone_Frequency,0.891588,Right (Positive)
SSTA,0.198355,Approximately Symmetric


In [30]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.preprocessing import OneHotEncoder

# Split first (no leakage)
X = df.drop(columns=['Percent_Bleaching'])
y = df['Percent_Bleaching']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Since we have categorical variables we need to seperate the numeric and the categorical variables
cat_cols = ['Realm_Name','Ocean_Name','Country_Name','Exposure','Bleaching_Level']
num_cols = [col for col in X.columns if col not in cat_cols]



# We one_hot_encode the categorical features
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

X_train_cat = ohe.fit_transform(X_train[cat_cols])
X_test_cat  = ohe.transform(X_test[cat_cols])


# We scale the numeric features
scaler = StandardScaler()

X_train_num = scaler.fit_transform(X_train[num_cols])
X_test_num  = scaler.transform(X_test[num_cols])


# Combine the categorical and numerical features
X_train_processed = np.hstack([X_train_num, X_train_cat])
X_test_processed  = np.hstack([X_test_num, X_test_cat])






In [31]:

import tensorflow as ts
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dropout


nodes_number = int((X_train_processed.shape[1] + 1) * 2/3)

model = Sequential()
model.add(Input(shape= (X_train_processed.shape[1],)))
model.add(Dense(nodes_number, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(nodes_number, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1))
model.summary()



Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_21 (Dense)                │ (None, 74)             │         8,214 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 74)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 74)             │         5,550 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 74)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 1)              │            75 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,839 (54.06 KB)

 Trainable params: 13,839 (54.06 KB)

 Non-trainable params: 0 (0.00 B)

Install cuda toolkit for GPU on desktop

Tensorflow GPU version (maybe try conda env)

In [32]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    patience=10,
    restore_best_weights=True,
    monitor='loss'
)

model.compile(optimizer="adam",
              loss="mse",
              metrics=['mae'],
               )

model.fit(X_train_processed, y_train, epochs=200, verbose=2, callbacks=[early_stop])

model.summary()

Epoch 1/200
818/818 - 4s - 5ms/step - loss: 49.4247 - mae: 5.8647
Epoch 2/200
818/818 - 2s - 3ms/step - loss: 42.5265 - mae: 5.2147
Epoch 3/200
818/818 - 2s - 3ms/step - loss: 41.1541 - mae: 5.0685
Epoch 4/200
818/818 - 2s - 3ms/step - loss: 40.3140 - mae: 4.9790
Epoch 5/200
818/818 - 3s - 3ms/step - loss: 39.3611 - mae: 4.9001
Epoch 6/200
818/818 - 2s - 3ms/step - loss: 38.8414 - mae: 4.8488
Epoch 7/200
818/818 - 2s - 3ms/step - loss: 38.3665 - mae: 4.8031
Epoch 8/200
818/818 - 2s - 3ms/step - loss: 38.0005 - mae: 4.7776
Epoch 9/200
818/818 - 2s - 3ms/step - loss: 37.3861 - mae: 4.7154
Epoch 10/200
818/818 - 2s - 3ms/step - loss: 37.0150 - mae: 4.6843
Epoch 11/200
818/818 - 2s - 3ms/step - loss: 36.7495 - mae: 4.6684
Epoch 12/200
818/818 - 2s - 3ms/step - loss: 36.3517 - mae: 4.6269
Epoch 13/200
818/818 - 2s - 3ms/step - loss: 36.0018 - mae: 4.5958
Epoch 14/200
818/818 - 2s - 3ms/step - loss: 35.7882 - mae: 4.5741
Epoch 15/200
818/818 - 2s - 3ms/step - loss: 35.4149 - mae: 4.5478
Epoc

KeyboardInterrupt: 

In [33]:
from sklearn.metrics import r2_score, mean_squared_error

y_pred = model.predict(X_test_processed)

score = r2_score(y_test, y_pred)

print(score)
print(mean_squared_error(y_test, y_pred))

205/205 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
0.5412287516986318
33.73150468039557


it takes absurd amount of time to make a very poor predict prediction, probably need to preprocess data again

Best score with such settings

nodes_number = int((X.shape[1] + 1) * 2/3)

model = Sequential()

model.add(Input(shape= (X.shape[1],)))

model.add(Dense(nodes_number, activation='relu'))

model.add(Dense(nodes_number, activation='relu'))

model.add(Dense(1))


with standard scaler data applied to both X and y

0.21329008825613271

0.7689129542072395

Probably it worth to try use 62 features with location and timestamps excluded. Could possibly apply PCA


In [8]:
from keras import Sequential

# -*- coding: utf-8 -*-
"""
Created on Wed Oct 29 15:40:04 2025

@author: zemsk


Bleaching Presence Detection
Target variable: Percent_Bleaching

"""

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, RobustScaler, StandardScaler
import os


path = "./coral-reef-global-bleaching"
filename_read = os.path.join(path, "coral.csv")

df = pd.read_csv(filename_read)

# # Removed only locations and labels, no real data touched
# df.drop("Ocean_Name", axis=1, inplace=True)
# df.drop("Country_Name", axis=1, inplace=True)
# df.drop("Sample_ID", axis=1, inplace=True)
# df.drop("Date_Year", axis=1, inplace=True)
# df.drop("Bleaching_Level", axis=1, inplace=True)
# df.drop("Realm_Name", axis=1, inplace=True)
#
# cols_to_drop = [
#     'Site_ID', 'Data_Source', 'Latitude_Degrees', 'Longitude_Degrees',
#     'Reef_ID', 'Ecoregion_Name', 'State_Island_Province_Name',
#     'City_Town_Name', 'Site_Name', 'Date', 'Date_Day', 'Date_Month',
#     'Site_Comments', 'Sample_Comments', 'Bleaching_Comments'
# ]
#
# df.drop(cols_to_drop, axis=1, inplace=True)
# #Percent_Cover is not a best predictor and also contain 30% of its fields as null.
# # For the sake of bigger dataset this feature is dropped
# df.drop("Percent_Cover", axis=1, inplace=True)
# df.drop("ClimSST", inplace=True, axis=1)
# df.drop("Exposure", inplace=True, axis=1)
# df.drop("Temperature_Maximum", inplace=True, axis=1)
#
# label_encoder = LabelEncoder()
# df["Exposure"] = label_encoder.fit_transform(df["Exposure"])

# These features were taken into account that data is nonlinear
#df = df[["Distance_to_Shore", "Temperature_Mean", "Turbidity", "TSA", "Depth_m", "Percent_Bleaching"]]

# These features were taken into account that data is linear
# df = df[['Cyclone_Frequency', 'Depth_m', 'ClimSST', 'Distance_to_Shore', 'Turbidity', 'TSA', 'Temperature_Mean', 'Percent_Bleaching']]

df.dropna(inplace=True)
df.info()



C:\Users\zemsk\AppData\Local\Temp\ipykernel_16180\3720938078.py:27: DtypeWarning: Columns (13,15,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename_read)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41361 entries, 0 to 41360
Data columns (total 62 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Site_ID                                41361 non-null  int64  
 1   Sample_ID                              41361 non-null  int64  
 2   Data_Source                            41361 non-null  object 
 3   Latitude_Degrees                       41361 non-null  float64
 4   Longitude_Degrees                      41361 non-null  float64
 5   Ocean_Name                             41361 non-null  object 
 6   Reef_ID                                41361 non-null  object 
 7   Realm_Name                             41361 non-null  object 
 8   Ecoregion_Name                         41361 non-null  object 
 9   Country_Name                           41361 non-null  object 
 10  State_Island_Province_Name             41361 non-null  object 
 11  Ci